In [1]:
#TFM UNIR BIG DATA

In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("/kaggle/input/dataset/preus_tancament_r.csv")

In [4]:
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Unnamed: 0,Date,GSPC.Open,GSPC.High,GSPC.Low,GSPC.Close,GSPC.Volume,GSPC.Adjusted,IBEX.Open,IBEX.High,...,CABK.Low,CABK.Close,CABK.Volume,CABK.Adjusted,FER.Open,FER.High,FER.Low,FER.Close,FER.Volume,FER.Adjusted
0,1,2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,3.991400e+09,1132.989990,11986.500000,12145.099609,...,1.37222,1.37222,0.0,1.37222,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,2.491020e+09,1136.520020,12141.799805,12240.500000,...,1.32222,1.36667,833.0,1.36667,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,4.972660e+09,1137.140015,12216.400391,12230.700195,...,1.36111,1.38333,163881.0,1.38333,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,5.270680e+09,1141.689941,12163.000000,12199.700195,...,1.36667,1.37778,31284.0,1.37778,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2010-01-08,1140.520020,1145.390015,1136.219971,1144.979980,4.389590e+09,1144.979980,12218.000000,12232.200195,...,1.37778,1.37778,0.0,1.37778,NaN,NaN,NaN,NaN,NaN,NaN
